# 数据抓取

## 1、抓取帖子列表

In [1]:
import urllib2
from bs4 import BeautifulSoup
def crawler(page_num, file_name):
    try:
        # open the browser
        url = "http://bbs.tianya.cn/list.jsp?item=develop&order=1&nextid=%d&k=gdp" % page_num
        content = urllib2.urlopen(url).read() 
        soup = BeautifulSoup(content, "lxml") 
        articles = soup.find_all('tr')
        # write down info
        for i in articles[1:]:
            td = i.find_all('td')
            title = td[0].text.strip()
            title_url = td[0].a['href']
            author = td[1].text
            author_url = td[1].a['href']
            views = td[2].text
            replies = td[3].text
            date = td[4]['title']
            record = title + '\t' + title_url+ '\t' + author + '\t'+ \
                        author_url + '\t' + views+ '\t'  + replies+ '\t'+ date
            with open(file_name,'a') as p: 
                        p.write(record.encode('utf-8')+"\n") ##!!encode here to utf-8 to avoid encoding

    except Exception, e:
        print e
        pass

for page_num in range(21):
    print (page_num)
    crawler(page_num,'C:/Users/lenovo/Desktop/finalproject/tianya_bbs_threads_list_gdp.txt')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


### 共抓取了天涯论坛上，“GDP”关键字下的20页帖子。

## 2、作者信息抓取

In [9]:
def author_crawler(url, file_name):
    try:
        content = urllib2.urlopen(url).read() #获取网页的html文本
        soup = BeautifulSoup(content, "lxml")
        link_info = soup.find_all('div', {'class', 'link-box'})
        followed_num, fans_num = [i.a.text for i in link_info]
        try:
            activity = soup.find_all('span', {'class', 'subtitle'})
            post_num, reply_num = [j.text[2:] for i in activity[:1] for j in i('a')]
        except:
            post_num, reply_num = 1, 0
        record =  '\t'.join([url, followed_num, fans_num, post_num, reply_num])
        with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                    p.write(record.encode('utf-8')+"\n") ##!!encode here to utf-8 to avoid encoding

    except Exception, e:
        print e, url
        record =  '\t'.join([url, 'na', 'na', 'na', 'na'])
        with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                    p.write(record.encode('utf-8')+"\n") ##!!encode here to utf-8 to avoid encoding
        pass

for k, url in enumerate(df.author_page) :
    if k%10==0:
        print k
    author_crawler(url,'C:/Users/lenovo/Desktop/finalproject/tianya_bbs_threads_gdp_author_info1.txt')

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
need more than 0 values to unpack http://www.tianya.cn/101980303
160
need more than 0 values to unpack http://www.tianya.cn/101532005
170
180
190
200
210
220
230
need more than 0 values to unpack http://www.tianya.cn/18383118
240
need more than 0 values to unpack http://www.tianya.cn/18383118
250
260
270
need more than 0 values to unpack http://www.tianya.cn/32922138
need more than 0 values to unpack http://www.tianya.cn/18383118
280
290
300
310
320
330
need more than 0 values to unpack http://www.tianya.cn/52188114
need more than 0 values to unpack http://www.tianya.cn/83108837
340
350
need more than 0 values to unpack http://www.tianya.cn/53582376
360
370
380
need more than 0 values to unpack http://www.tianya.cn/88640588
need more than 0 values to unpack http://www.tianya.cn/88640588
390
need more than 0 values to unpack http://www.tianya.cn/88640588
need more than 0 values to unpack http://www.tianya.cn/88640588
need more than 0 

### 共抓取了1040位作者的信息。

## 3、回帖抓取

In [11]:
import urllib2
from bs4 import BeautifulSoup
def parsePage(pa):
    records = []
    for i in pa:
        p_info = i.find('a', class_ = 'reportme a-link')
        p_time = p_info['replytime']
        p_author_id = p_info['authorid']
        p_author_name = p_info['author']
        p_content = i.find('div', {'class', 'bbs-content'}).text.strip()
        p_content = p_content.replace('\t', '').replace('\n', '')#.replace(' ', '')
        record = p_time + '\t' + p_author_id+ '\t' + p_author_name + '\t'+ p_content
        records.append(record)
    return records
import sys
def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()


In [12]:
import random
import time

def crawler(url, file_name):
    try:
        # open the browser
        url_1 = 'http://bbs.tianya.cn' + url
        content = urllib2.urlopen(url_1).read() #获取网页的html文本
        post_soup = BeautifulSoup(content, "lxml") 
        # how many pages in a post
        post_form = post_soup.find('div', {'class', 'atl-pages'})
        if post_form.form:
            post_pages = post_form.form['onsubmit'].split(',')[-1].split(')')[0]
            post_pages = int(post_pages)
            url_base = '-'.join(url_1.split('-')[:-1]) + '-%d.shtml'
        else:
            post_pages = 1
        # for the first page
        pa = post_soup.find_all('div', {'class', 'atl-item'})
        records = parsePage(pa)
        with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
            for record in records:    
                p.write('1'+ '\t' + url + '\t' + record.encode('utf-8')+"\n") 
        # for the 2nd+ pages
        if post_pages > 1:
            for page_num in range(2, post_pages+1):
                time.sleep(random.random())
                flushPrint(page_num)
                url2 =url_base  % page_num
                content = urllib2.urlopen(url2).read() #获取网页的html文本
                post_soup = BeautifulSoup(content, "lxml") 
                pa = post_soup.find_all('div', {'class', 'atl-item'})
                records = parsePage(pa)
                with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                    for record in records:    
                        p.write(str(page_num) + '\t' +url + '\t' + record.encode('utf-8')+"\n") 
        else:
            pass
    except Exception, e:
        print e
        pass


In [14]:
url = 'http://bbs.tianya.cn' + df.link[2]
file_name = 'C:/Users/lenovo/Desktop/finalproject/tianya_bbs_threads_test.txt'
crawler(url, file_name)

<urlopen error [Errno 11004] getaddrinfo failed>


In [15]:
for k, link in enumerate(df.link):
    flushPrint(link)
    if k % 10== 0:
        print 'This it the post of : ' + str(k)
    file_name = 'C:/Users/lenovo/Desktop/finalproject/tianya_bbs_threads_network.txt'
    crawler(link, file_name)

/post-develop-2142230-1.shtmlThis it the post of : 0
/post-develop-2132776-1.shtmlThis it the post of : 10
/post-develop-2125152-1.shtmlThis it the post of : 20
/post-develop-2117648-1.shtmlThis it the post of : 30
/post-develop-2112589-1.shtmlThis it the post of : 40
/post-develop-2112061-1.shtmlThis it the post of : 50
/post-develop-2101547-1.shtmlThis it the post of : 60
/post-develop-2096826-1.shtmlThis it the post of : 70
/post-develop-2092286-1.shtmlThis it the post of : 80
/post-develop-2088967-1.shtmlThis it the post of : 90
/post-develop-2085565-1.shtmlThis it the post of : 100
/post-develop-2083335-1.shtmlThis it the post of : 110
/post-develop-2068176-1.shtmlThis it the post of : 120
/post-develop-2060087-1.shtmlThis it the post of : 130
/post-develop-2057048-1.shtmlThis it the post of : 140
/post-develop-2049441-1.shtmlThis it the post of : 150
/post-develop-2029595-1.shtmlThis it the post of : 160
/post-develop-2024061-1.shtmlThis it the post of : 170
/post-develop-2019567

### 抓取了1040个帖子的所有网友回复。